# Curve fitting

- [LsqFit.jl](https://github.com/JuliaNLSolvers/LsqFit.jl)
- [NonlinearSolve.jl](https://github.com/SciML/NonlinearSolve.jl)
- [CurveFit.jl](https://github.com/SciML/CurveFit.jl)

## LsqFit

`LsqFit.jl` package is a small library that provides basic least-squares fitting in pure Julia.

In [ ]:
using LsqFit
@. model(x, p) = p[1] * exp(-x * p[2])

# Generate data
xdata = range(0, stop=10, length=20)
ydata = model(xdata, [1.0 2.0]) + 0.01 * randn(length(xdata))
# Initial guess
p0 = [0.5, 0.5]

# Fit the model
@time fit = curve_fit(model, xdata, ydata, p0; autodiff=:forwarddiff)

# The result should be close to `[1.0 2.0]`
coef(fit)

## NonlinearSolve

The algorithms for `NonlinearLeastSquaresProblem` in `NonlinearSolve.jl` are better suited for ill-conditioned nonlinear systems as stated in [JuliaCon 2025](https://youtu.be/mdcCjaYSNNc)

The upcoming `CurveFit.jl` uses `NonlinearSolve.jl`.

In [ ]:
using NonlinearSolve

model(x, p) = @. p[1] * exp(-x * p[2])

# Generate data
xdata = range(0, stop=10, length=20)
ydata = model(xdata, [1.0 2.0]) + 0.01 * randn(length(xdata));

In [ ]:
function nonlinearfit(model, p0, xdata, ydata; opts...)
    ## Residual function
    function resid_func(p, data)
        x = data[1]
        y = data[2]
        ypred = model(x, p)
        return ypred .- y
    end

    nlls_prob = NonlinearLeastSquaresProblem(resid_func, p0, (xdata, ydata))
    sol = solve(nlls_prob; opts...)
    resid = abs2.(resid_func(sol.u, (xdata, ydata))) |> sum
    rmse = sqrt(resid / length(ydata))
    return (;sol, rmse)
end

In [ ]:
# Initial guess
p0 = [0.0 0.0]

# Fit the model. The result should be close to `[1.0 2.0]`
@time (; sol, rmse) = nonlinearfit(model, p0, xdata, ydata; maxiters = 1000)